<a href="https://colab.research.google.com/github/pauchai/SubPlayer/blob/master/Paul%20Himyak/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

from tensorflow.keras.preprocessing.text import Tokenizer

import gdown

In [44]:
dataset_url = "https://docs.google.com/spreadsheets/d/1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg/edit?usp=share_link&ouid=113369956649761615778&rtpof=true&sd=true"
dataset_filename = "profpatolog_modified.xlsx"
#!wget -qc dataset_url -O dataset_filename
gdown.download(dataset_url, output=dataset_filename, quiet=True )


/usr/local/lib/python3.9/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(


'profpatolog_modified.xlsx'

In [47]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
df = pd.read_excel("/content/drive/MyDrive/profpatolog_modified.xlsx")


In [50]:
#Удаляем случайные вбросы
df.drop([843, 851, 1022, 1024], inplace = True)

#Убираем пустые колонки
empty_columns = df.columns[df.isna().all()]
df.drop(empty_columns, inplace = True, axis = 1)

In [51]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 0
FIELD_CONCLUSION_PART_FIT = 1
FIELD_CONCLUSION_NOT_FIT = 2

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"

In [53]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns


In [96]:
df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: x.replace(".", "x"))

In [114]:
df

,Unnamed: 0,КлиентДатаРождения,КлиентПол,ЗаключениеМК,ДиспансернаяГруппа,ВредныеФакторы,ПсихиатрияНаркология1_МКБ101,Офтальмология1_МКБ101,Оториноларингология1_МКБ101,ХирургНеИсп1_МКБ101,...,Психиатрия1_МКБ102,Психиатрия2_МКБ101,Психиатрия2_МКБ102,Рентгенология1_МКБ102,Рентгенология2_МКБ102,Психиатрия3_МКБ101,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_МКБ102,Оториноларингология3_МКБ101
0,0,1976-10-24,Женский,Годен,I,[11x4],Z00.0,Z00.0,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1987-11-18,Мужской,Годен,IIIа,[4x2x5],Z00.0,Z00.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1985-09-08,Мужской,ГоденСКоррекциейЗрения,II,[4x2x5],Z00.0,H52.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1988-04-25,Мужской,Годен,I,[11x1],NaN,Z00.0,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1984-04-17,Мужской,ГоденСКоррекциейЗрения,II,[11x1],NaN,H52.2,Z00.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,1977,1972-11-12,Мужской,NaN,NaN,[6x1],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978,1978,1986-07-09,Мужской,NaN,NaN,[6x1],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,1979,1994-05-12,Мужской,NaN,NaN,"[5x1, 23]",Z00.0,Z00.0,Z00.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,1980,1972-04-17,Женский,NaN,NaN,[23],Z00.0,NaN,Z00.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df[FIELD_HARMFUL_FACTORS]

0         11x4
1        4x2x5
2        4x2x5
3         11x1
4         11x1
         ...  
1977       6x1
1978       6x1
1979    5x1,23
1980        23
1981        27
Name: ВредныеФакторы, Length: 1978, dtype: object

In [107]:
df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].str.split(",")

In [108]:
harmful_values = df[FIELD_HARMFUL_FACTORS].explode().sort_values().unique()

In [109]:
harmful_values

array(['10', '11', '11x1', '11x2', '11x3', '11x4', '12', '13', '14', '15',
       '17', '18', '18x1', '18x2', '1x1', '1x14x1', '1x14x2', '1x15',
       '1x17', '1x18x3', '1x19x1', '1x2', '1x23', '1x24', '1x29',
       '1x29x1', '1x29x2', '1x29x4', '1x3', '1x30x1', '1x34x1', '1x36',
       '1x36x1', '1x37', '1x37x1', '1x37x1x1', '1x37x1x2', '1x39',
       '1x45x1', '1x46', '1x48', '1x49', '1x49x14', '1x49x2', '1x50',
       '1x8x1', '1x8x1x1', '1x8x1x2', '1x8x2', '1x8x2x1', '1x8x2x2',
       '1x8x3', '1x8x3x1', '1x8x3x2', '22', '23', '24', '25', '26', '27',
       '2x4', '2x4x3', '3x1', '3x1x2', '3x1x7', '3x1x8x1', '3x1x8x2',
       '3x1x8x3', '3x4', '4x1', '4x10', '4x2', '4x2x1', '4x2x3', '4x2x4',
       '4x2x5', '4x3x1', '4x3x2', '4x4', '4x5', '4x6', '4x7', '4x8',
       '4x9', '5x1', '5x2', '5x2x1', '6', '6x1', '6x2', '8', '9'],
      dtype=object)

In [110]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='unknown', char_level=False)
tokenizer.fit_on_texts(harmful_values)

In [115]:
df[FIELD_HARMFUL_FACTORS][1979]

['5x1', '23']

In [123]:
tokenizer.texts_to_sequences("5x1 23")

[[1], [1], [1], [], [1], [1]]

In [106]:
" ".join(['1x29x1', '1x29x2'])

'1x29x1 1x29x2'

In [121]:
tokenized

[[86], [57]]

In [118]:
np.ravel(tokenized)

array([86, 57])